In [1]:
import numpy as np
import pythonToC
import parsing_yacc 
from parsing_lex import MyLexer
import pythonTopython2
import time
import pandas as pd

def measure_execution_time(input_code, mode):
    start_time = time.time()
    output = deploy_parser(input_code, mode)
    execution_time = time.time() - start_time
    return output, execution_time

def deploy_parser(input, mode): 
    output = ""

    for line in input.split("\n"):
        if line == "":
            continue
        result = parsing_yacc.main(line, mode)
        if isinstance(result, str):
            output += result + "\n"

    print(output)
    return output

def create_input_code(arr_len): 
    # Create a list of values with length arr_len 
    a_values = np.random.randint(1, 100, arr_len)
    a_values = a_values.tolist() 
    b_values = np.random.randint(1, 100, arr_len) 
    b_values = b_values.tolist() 

    # Create a string with the code to be executed
    # print(str(a_values)[1:-1])
    code = """
a = np.array([%s]) 
b = np.array([%s])

c = np.add(a, b) 
d = np.subtract(a, b)
e = np.sum(a) 
f = np.dot(a, b)""" % (str(a_values)[1:-1], str(b_values)[1:-1])

    return code

def run_trials(trials, array_lengths):
    results = []
    for trial in range(trials):
        for array_length in array_lengths:
            input_code = create_input_code(array_length)
            c_output, c_time = measure_execution_time(input_code, "C")
            python_output, python_time = measure_execution_time(input_code, "Python")
            results.append((trial, array_length, c_time, python_time))
    return results

# Create an instance of MyLexer
lexer = MyLexer()

# Build the lexer
lexer.build()

# Test the lexer on the input code
input_code = """
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

c = np.add(a, b)
d = np.subtract(a, b)
e = np.sum(a)
f = np.dot(a, b)
"""
lexer.test(input_code)

trials = 3
array_lengths = [5, 10, 50, 100, 500, 1000]

results = run_trials(trials, array_lengths)

# print(results)

# # Print the results
# for result in results:
#     print("Trial:", result[0])
#     print("Array Length:", result[1])
#     print("C Code Generation Time:", result[2])
#     print("Python Code Generation Time:", result[3])
#     print()

# Construct a DataFrame from results 
results_df = pd.DataFrame(results, columns=["Trial", "Array Length", "C Code Generation Time", "Python Code Generation Time"])

results_df.to_csv('results.csv', index=False)

display(results_df)


LexToken(VARIABLE,'a',1,1)
LexToken(EQUALS,'=',1,3)
LexToken(ARRAY,'np.array',1,5)
LexToken(LPAREN,'(',1,13)
LexToken(LIST,'[1, 2, 3]',1,14)
LexToken(RPAREN,')',1,23)
LexToken(VARIABLE,'b',1,25)
LexToken(EQUALS,'=',1,27)
LexToken(ARRAY,'np.array',1,29)
LexToken(LPAREN,'(',1,37)
LexToken(LIST,'[4, 5, 6]',1,38)
LexToken(RPAREN,')',1,47)
LexToken(VARIABLE,'c',1,50)
LexToken(EQUALS,'=',1,52)
LexToken(NP_ADD,'np.add',1,54)
LexToken(LPAREN,'(',1,60)
LexToken(VARIABLE,'a',1,61)
LexToken(COMMA,',',1,62)
LexToken(VARIABLE,'b',1,64)
LexToken(RPAREN,')',1,65)
LexToken(VARIABLE,'d',1,67)
LexToken(EQUALS,'=',1,69)
LexToken(NP_SUBTRACT,'np.subtract',1,71)
LexToken(LPAREN,'(',1,82)
LexToken(VARIABLE,'a',1,83)
LexToken(COMMA,',',1,84)
LexToken(VARIABLE,'b',1,86)
LexToken(RPAREN,')',1,87)
LexToken(VARIABLE,'e',1,89)
LexToken(EQUALS,'=',1,91)
LexToken(NP_SUM,'np.sum',1,93)
LexToken(LPAREN,'(',1,99)
LexToken(VARIABLE,'a',1,100)
LexToken(RPAREN,')',1,101)
LexToken(VARIABLE,'f',1,103)
LexToken(EQUALS,'=',1

,Trial,Array Length,C Code Generation Time,Python Code Generation Time
0,0,5,0.020026,0.000000
1,0,10,0.006153,0.002858
2,0,50,0.006492,0.004961
3,0,100,0.004251,0.003390
4,0,500,0.005842,0.000000
5,0,1000,0.011462,0.004324
6,1,5,0.004608,0.003793
7,1,10,0.005356,0.000000
8,1,50,0.000000,0.013295
9,1,100,0.003309,0.000000
